In [1]:
import matplotlib.pyplot as plt 
from matplotlib.pyplot import cm
import numpy as np
import pickle

%matplotlib

Using matplotlib backend: <object object at 0x7f6aa99859e0>


# View times in a dataframe

In [14]:
import pandas as pd

# unpickle times and states
filename = "isothermal_idaklu.pkl"
with open("data/" + filename, 'rb') as handle:
    times_and_states = pickle.load(handle)

# create dataframe (columns are labelled by number of coupled DFNs)
# TODO: unpack list of solve times
df = pd.DataFrame(times_and_states)
df

,1,3,4,8,16,32,64,128
states,641,1927,2569,5137,10273,20545,41089,82177
setup times,[0.13938977600628277],[0.27479812200181186],[0.3155859109974699],[0.5497924330047681],[1.023060722000082],[2.0061546789947897],[4.191727546000038],[9.109540707002452]
solve times,[0.19728467300592456],[0.5128933830055757],[0.7372321709990501],[1.4079110020029475],[2.767006260000926],[5.9560455859973445],[12.886227426002733],[25.608393064001575]
integration times,[0.19073901099909563],[0.5008638319995953],[0.7232005989935715],[1.3859029219966033],[2.7284670720036956],[5.880837563003297],[12.73332346299867],[25.30394251600228]
setup time,0.13939,0.274798,0.315586,0.549792,1.023061,2.006155,4.191728,9.109541
average solve time,0.197285,0.512893,0.737232,1.407911,2.767006,5.956046,12.886227,25.608393
average integration time,0.190739,0.500864,0.723201,1.385903,2.728467,5.880838,12.733323,25.303943


# Plot times

Here we define a function to create a plot of setup time, average solve time and average integration time vs number of coupled DFN models, and a loglog plot of integration time vs number of states.

In [3]:
def plot_times(filename, title=None):
    "Plot times for a given filename"
    
    # unpickle times and make lists
    with open("data/" + filename, 'rb') as handle:
        times_and_states = pickle.load(handle)

    npts = list(times_and_states.keys())
    states = [times_and_states[i]["states"] for i in npts]
    setup_times = [times_and_states[i]["setup time"] for i in npts]
    solve_times = [times_and_states[i]["average solve time"] for i in npts]
    integration_times = [times_and_states[i]["average integration time"] for i in npts]

    # create figure
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    title = title or filename[:-4]
    
    # add times to plot
    ax[0].plot(npts, setup_times, "-o", label="Setup time")
    ax[0].plot(npts, solve_times, "-o", label="Solve time")
    ax[0].plot(npts, integration_times, "-o", label="Integration time")
    ax[1].loglog(states, integration_times, "-o")

    # add fit to states vs integration times 
    m, c = np.polyfit(np.log10(states), np.log10(integration_times), 1)
    ax[1].loglog(states, 10**c * states**m, "-")
    ax[1].text(1e3, 10, f"slope = {np.round(m,2)}")

    # add labels
    ax[0].set_xlabel("Number of coupled DFN models")
    ax[0].set_ylabel("Time [s]")            
    ax[0].legend()
    ax[1].set_xlabel("Number of states")
    ax[1].set_ylabel("Integration time [s]")        

    plt.suptitle(title)
    plt.tight_layout()
        

In [4]:
plot_times("thermal_idaklu.pkl")

# Compare times

Here we define a function to create a plot to compare setup times, average solve times and average integration times from one or more files.

In [5]:
def compare_times(filenames, labels=None):
    """Compare solve times for different files"""
    
    # create figure
    fig, ax = plt.subplots(2, 2, figsize=(8, 8))
    labels = labels or [filename[:-4] for filename in filenames]
    colors = iter(cm.tab10(np.linspace(0, 1, 10)))
    n = 0
    
    for filename, label in zip(filenames, labels):
        color = next(colors)
        n = n+1
        
        # unpickle times and make lists
        with open("data/" + filename, 'rb') as handle:
            times_and_states = pickle.load(handle)
            
        npts = list(times_and_states.keys())
        states = [times_and_states[i]["states"] for i in npts]
        setup_times = [times_and_states[i]["setup time"] for i in npts]
        solve_times = [times_and_states[i]["average solve time"] for i in npts]
        integration_times = [times_and_states[i]["average integration time"] for i in npts]

        # add times to plot
        ax[0, 0].plot(npts, setup_times, "-o", c=color)
        ax[0, 1].plot(npts, solve_times, "-o", c=color)
        ax[1, 0].plot(npts, integration_times, "-o", c=color)
        ax[1, 1].loglog(states, integration_times, "-o", label=label, c=color)

    # add labels
    ax[0, 0].set_xlabel("Number of coupled DFN models")
    ax[0, 0].set_ylabel("Setup time [s]")   
    ax[0, 1].set_xlabel("Number of coupled DFN models")
    ax[0, 1].set_ylabel("Solve time [s]")        
    ax[1, 0].set_xlabel("Number of coupled DFN models")
    ax[1, 0].set_ylabel("Integration time [s]")       
    ax[1, 1].set_xlabel("Number of states")
    ax[1, 1].set_ylabel("Integration time [s]")        
    if len(filenames) > 1:
        ax[1, 1].legend(loc="lower right")

    plt.tight_layout()
    
    return fig, ax

In [7]:
%matplotlib
filenames = ["thermal_idaklu.pkl", "thermal_casadi.pkl", "isothermal_idaklu.pkl", "isothermal_casadi.pkl"]
_, ax = compare_times(filenames)

Using matplotlib backend: TkAgg


## compare solver and thermal option

In [10]:
def compare(filenames, labels=None):
    """Compare solve times for different files"""
    
    # create figure
    fig, ax = plt.subplots(2, 2, figsize=(8, 8))
    labels = labels or [filename[:-4] for filename in filenames]
    
    for filename, label in zip(filenames, labels):
        if "idaklu" in filename:
            color = "blue"
        else:
            color = "red"
        if "isothermal" in filename:
            linestyle = "dashed"
        else:
            linestyle = "solid"
            
        
        # unpickle times and make lists
        with open("data/" + filename, 'rb') as handle:
            times_and_states = pickle.load(handle)
            
        npts = list(times_and_states.keys())
        states = [times_and_states[i]["states"] for i in npts]
        setup_times = [times_and_states[i]["setup time"] for i in npts]
        solve_times = [times_and_states[i]["average solve time"] for i in npts]
        integration_times = [times_and_states[i]["average integration time"] for i in npts]

        # add times to plot
        ax[0, 0].plot(npts, setup_times, linestyle=linestyle, c=color)
        ax[0, 1].plot(npts, solve_times, linestyle=linestyle, c=color)
        ax[1, 0].plot(npts, integration_times, linestyle=linestyle, c=color)
        ax[1, 1].loglog(states, integration_times, linestyle=linestyle, label=label, c=color)

    # add slope
    ax[1, 1].loglog(states, np.array(states) / 500, "-")
    ax[1, 1].text(1e3, 10, "slope = 1")

    # add labels
    ax[0, 0].set_xlabel("Number of coupled DFN models")
    ax[0, 0].set_ylabel("Setup time [s]")   
    ax[0, 1].set_xlabel("Number of coupled DFN models")
    ax[0, 1].set_ylabel("Solve time [s]")        
    ax[1, 0].set_xlabel("Number of coupled DFN models")
    ax[1, 0].set_ylabel("Integration time [s]")       
    ax[1, 1].set_xlabel("Number of states")
    ax[1, 1].set_ylabel("Integration time [s]")        
    if len(filenames) > 1:
        ax[1, 1].legend(loc="lower right")

    plt.tight_layout()
    
    return fig, ax

In [11]:
%matplotlib
filenames = ["thermal_idaklu.pkl", "isothermal_idaklu.pkl", "thermal_casadi.pkl", "isothermal_casadi.pkl"]
_, ax = compare(filenames)

Using matplotlib backend: TkAgg
